Harshvardhan, 20D070035, Assignment 3 for EE679

In [ ]:
import librosa
import librosa.display
import librosa.display
import matplotlib.pyplot as plt
# Import the matplotlib.pyplot module for creating plots and visualizations.
import numpy as np
import os
from hmmlearn import hmm
import shutil
from zipfile import ZipFile
# Import the ZipFile class from zipfile for working with zip files.
from zipfile import ZipFile
# Import various metrics for evaluating machine learning models.
from sklearn.metrics import confusion_matrix

In [ ]:
source_zip_file = r'D:\CA3-ASR.zip'
destination_directory = r'D:\EE679_Assignment3'

shutil.unpack_archive(source_zip_file, extract_dir=destination_directory)

In [ ]:
# Source ZIP file on your local machine (Windows)
source_zip_file = r'D:\EE679_Assignment3\CA3-ASR\Commands Dataset-20200305T135856Z-001.zip'

# Destination directory on your local machine (Windows)
destination_directory = r'D:\EE679_Assignment3\Assignment3'

# Extracting the contents of the ZIP file
with ZipFile(source_zip_file, 'r') as zip_ref:
    zip_ref.extractall(destination_directory)

# Renaming directories as per your requirements
original_train_path = os.path.join(destination_directory, 'Commands Dataset', 'train')
new_train_zip_path = os.path.join(destination_directory, 'Commands Dataset', 'train_zip')

# Check if the destination directory exists before renaming
if os.path.exists(new_train_zip_path):
    print(f"Destination directory '{new_train_zip_path}' already exists.")
else:
    os.rename(original_train_path, new_train_zip_path)

# Navigating to the Commands_Dataset directory
os.chdir(os.path.join(destination_directory, 'Commands Dataset'))

# Listing the contents of the current directory
print(os.listdir())

# Unzipping background noise dataset
shutil.unpack_archive("_background_noise_.zip", extract_dir=os.path.join(destination_directory, 'Commands Dataset'))

# Unzipping test datasets
shutil.unpack_archive("test_clean.zip", extract_dir=os.path.join(destination_directory, 'Commands Dataset'))
shutil.unpack_archive("test_noisy.zip", extract_dir=os.path.join(destination_directory, 'Commands Dataset'))


In [ ]:
import os
import shutil

def extract_and_set_permissions(zip_path, extract_path):
    # Check if the destination directory already exists
    if not os.path.exists(extract_path):
        os.makedirs(extract_path)

    # Iterate through files in the ZIP archive
    for root, dirs, files in os.walk(zip_path):
        for filename in files:
            name = filename.split(".")[0]
            path_zip = os.path.join(zip_path, name)
            path_unzip = os.path.join(extract_path, name)

            # Check if the file or directory already exists
            if not os.path.exists(path_unzip):
                # Extract the archive to the destination directory
                shutil.unpack_archive(path_zip + ".zip", extract_dir=path_unzip)

                # Set permissions for the extracted files and directories (adjust as needed)
                for dirpath, dirnames, filenames in os.walk(path_unzip):
                    os.chmod(dirpath, 0o755)  # Example: set directory permissions to rwxr-xr-x
                    for filename in filenames:
                        filepath = os.path.join(dirpath, filename)
                        os.chmod(filepath, 0o644)  # Example: set file permissions to rw-r--r--

# Example usage
train_zip_path = "D:\\EE679_Assignment3\\Assignment3\\Commands Dataset\\train"
train_path = "D:\\EE679_Assignment3\\Assignment3\\Commands Dataset\\extracted_train"

extract_and_set_permissions(train_zip_path, train_path)
print(os.listdir())


In [ ]:
def train_test_read(datapath, dataset_array):
    for filename in os.listdir(datapath):
        filepath = os.path.join(datapath, filename)

        try:
            if os.path.isfile(filepath):
                sound_file, sr = librosa.load(filepath)
                dataset_array.append([sound_file, sr, filename])
                print(filepath)
            elif os.path.isdir(filepath):
                for sample in os.listdir(filepath):
                    sample_file_path = os.path.join(filepath, sample)
                    try:
                        sound_file, sr = librosa.load(sample_file_path)
                        dataset_array.append([sound_file, sr, filename])
                        print(sample_file_path)
                    except Exception as e:
                        print(f"Error loading {sample_file_path}: {e}")
        except Exception as e:
            print(f"Error processing {filepath}: {e}")

In [ ]:
# Ensure the train_clean directory exists
test_clean_path = "D:\\EE679_Assignment3\\Assignment3\\Commands Dataset\\test_clean"
# try:
#     os.mkdir(train_clean_path)
# except FileExistsError:
#     pass

In [ ]:
# Ensure the train_noisy directory exists
test_noisy_path = "D:\\EE679_Assignment3\\Assignment3\\Commands Dataset\\test_noisy"
# try:
#     os.mkdir(train_noisy_path)
# except FileExistsError:
#     pass

Definations:

In [ ]:
def pre_emp(input,k):
  duration = len(input) # Calculate the duration (length) of the input signal
  output = np.zeros_like(input) # Create an array to store the pre-emphasized signal, initialized with zeros
  output[0] = input[0] # Create an array to store the pre-emphasized signal, initialized with zeros
  for n in range(1,duration): # Loop through the input signal to perform pre-emphasi
    output[n] = input[n] - k*input[n-1]  # Apply the pre-emphasis formula: y[n] = x[n] - k * x[n-1]
  return output

In [ ]:
def end_pointing(sound_file, sampling_rate):
    # Identify endpoints in the signal based on energy in 20ms window frames

    # Calculate the size of the window in samples (20 milliseconds)
    window_size = int(sampling_rate * 0.01)

    # Divide the sound file into overlapping frames of the specified window size
    frames = [sound_file[i * window_size : (i + 2) * window_size] for i in range(len(sound_file) // window_size - 1)]

    # Initialize variables for start and end points, noise flag, and update flag
    start, end, is_noise, is_updated = 0, 0, True, False

    # Calculate the Hamming window to be used for frame energy calculation
    hamming_window = np.hamming(window_size * 2)

    # Iterate through each frame to identify start and end points
    for i, frame in enumerate(frames):
        # Calculate energy of the frame using Hamming window
        energy = np.matmul(frame * hamming_window, frame * hamming_window)

        # Check if energy is above a threshold and update start point
        if energy > 0.005:
            if is_noise:
                is_noise = False
                if not is_updated:
                    is_updated = True
                    start = i

        # Check if energy is below a threshold and update end point
        if energy < 0.00001 and not is_noise:
            is_noise = True
            end = i
            break

    # Calculate the duration and adjust the end point if the duration is too short
    duration = end - start
    end = len(frames) - 1 if duration < 50 else end

    # Extract the portion of the sound file between identified start and end points
    new_sound_file = np.array(sound_file[start * window_size : (end + 2) * window_size])

    # Return the modified sound file
    return new_sound_file


In [ ]:
def mfcc(sound_file, sr):
    # Compute MFCC features along with first and second-order deltas

    # Compute the MFCC features using librosa with 13 coefficients
    mfcc = librosa.feature.mfcc(y=sound_file, sr=sr, n_mfcc=13, dct_type=2, norm='ortho')

    # Compute the first-order delta of the MFCC features
    mfcc_delta = librosa.feature.delta(mfcc, order=1, mode='nearest')

    # Compute the second-order delta of the MFCC features
    mfcc_delta2 = librosa.feature.delta(mfcc, order=2, mode='nearest')

    # Concatenate the MFCC features, first-order deltas, and second-order deltas along the vertical axis
    mfcc_features = np.concatenate((mfcc, mfcc_delta, mfcc_delta2), axis=0)

    # Return the concatenated MFCC features
    return mfcc_features


In [ ]:
def prediction(test_data, trained_model):
    # Calculate the score values for each word in the trained model
    score_values = {word: trained_model[word].score(test_data) for word in trained_model.keys()}

    # Determine the predicted label based on the word with the maximum score
    predicted_label = max(score_values, key=score_values.get)

    # Return the predicted label
    return predicted_label


In [ ]:
def plot_confusion_matrix(cm, classes, title, filename):
    # Set the color map for the plot
    cmap = plt.cm.YlOrBr

    # Plot the confusion matrix
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)

    # Set tick marks and labels for axes
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    # Adjust layout for better visualization
    plt.tight_layout()

    # Set labels for axes
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

    # Add color bar to the plot
    plt.colorbar()

    # Save the plot to the specified file
    plt.savefig(filename)

    # Display the plot
    plt.show()


Train-Test begins:

In [ ]:
train_path = r"D:\EE679_Assignment3\Assignment3\Commands Dataset\train file"
labels = ["down", "go", "left", "no", "off", "on", "right", "stop", "up", "yes"]
# # Extracting contents of train_zip directory to train directory
# for root, dirs, files in os.walk(train_zip_path):
#     for filename in files:
#         name = filename.split(".")[0]
#         path_zip = os.path.join(train_zip_path, name)
#         path_unzip = os.path.join(train_path, name)
#         shutil.unpack_archive(path_zip + ".zip", extract_dir=path_unzip)
dataset = {label: [] for label in labels}
import wave
for label in labels:
    folder = os.path.join(train_path, label)
    files = [x for x in os.listdir(folder) if x.endswith(".wav")][:2000]
    for file in files:
        dataset[label].append(wav.read(os.path.join(folder, file))[1])

In [ ]:
# Specify paths for test_clean, test_noisy, and _background_noise_ directories
test_noisy_path = "D:\\EE679_Assignment3\\Assignment3\\Commands Dataset\\test_noisy"
noise_path = r"D:\EE679_Assignment3\Assignment3\Commands Dataset\_background_noise_"

# Reading datasets
train_dataset = []
test_clean_dataset = []
test_noisy_dataset = []
noise_dataset = []

train_test_read(train_path, train_dataset)
train_test_read(test_noisy_path, test_noisy_dataset)
train_test_read(noise_path, noise_dataset)

In [ ]:
train_data = {}

# Loop through each item in the training dataset
for sound_file, sr, word in train_dataset:
    # Apply endpointing to the sound file
    processed_sound = end_pointing(sound_file, sr)
    
    # Check if the length of the processed sound is greater than 10000
    if len(processed_sound) > 10000:
        # Apply pre-emphasis to the processed sound
        processed_sound = pre_emp(processed_sound)  

        # Compute MFCC features and transpose the result
        mfcc_features = mfcc(processed_sound, sr).T
        
        # Append the MFCC features to the corresponding word in the train_data dictionary
        train_data.setdefault(word, []).append(mfcc_features)


In [ ]:
test_clean_data = {}

# Iterate through each word in the test clean path
for word in os.listdir(test_clean_path):
    test_clean_folder = os.path.join(test_clean_path, word)

    # Iterate through each sample in the current word folder
    for sample in os.listdir(test_clean_folder):
        test_clean_sample_path = os.path.join(test_clean_folder, sample)

        # Load the sound file and its sampling rate using librosa
        sound_file, sr = librosa.load(test_clean_sample_path)

        # Apply endpointing to the sound file
        processed_sound = end_pointing(sound_file, sr)

        # Check if the length of the processed sound is greater than 10000
        if len(processed_sound) > 10000:
            # Apply pre-emphasis to the processed sound
            processed_sound = pre_emp(processed_sound, k=0.97)  # Provide the value for k

            # Compute MFCC features and transpose the result
            mfcc_features = mfcc(processed_sound, sr).T

            # Append the MFCC features to the corresponding word in the test_clean_data dictionary
            test_clean_data.setdefault(word, []).append(mfcc_features)


In [ ]:
test_noisy_data = {}

# Iterate through each word in the test noisy path
for word in os.listdir(test_noisy_path):
    test_noisy_folder = os.path.join(test_noisy_path, word)

    # Iterate through each sample in the current word folder
    for sample in os.listdir(test_noisy_folder):
        test_noisy_sample_path = os.path.join(test_noisy_folder, sample)

        # Load the sound file and its sampling rate using librosa
        sound_file, sr = librosa.load(test_noisy_sample_path)

        # Apply endpointing to the sound file
        processed_sound = end_pointing(sound_file, sr)

        # Check if the length of the processed sound is greater than 10000
        if len(processed_sound) > 10000:
            # Apply pre-emphasis to the processed sound with k=0.97
            processed_sound = pre_emp(processed_sound, k=0.97)

            # Compute MFCC features and transpose the result
            mfcc_features = mfcc(processed_sound, sr).T

            # Append the MFCC features to the corresponding word in the test_noisy_data dictionary
            test_noisy_data.setdefault(word, []).append(mfcc_features)


In [ ]:
print(f"Number of samples in train dataset: {len(train_dataset)}")
print(f"Number of samples in test_clean dataset: {len(test_clean_dataset)}")
print(f"Number of samples in test_noisy dataset: {len(test_noisy_dataset)}")

In [ ]:
trained_model = {}
hmm_states = 9

# Iterate through each word and its corresponding training data
for word, current_training_data in train_data.items():
    print(word)
    
    # Initialize a Gaussian HMM model with the specified number of states
    model = hmm.GaussianHMM(n_components=hmm_states, covariance_type='full', n_iter=7)
    
    # Calculate the lengths of each training sequence
    lengths = np.array([data.shape[0] for data in current_training_data])
    
    # Stack the training data vertically to create a single array
    current_training_data = np.vstack(current_training_data)
    
    # Fit the model to the training data with the specified sequence lengths
    model.fit(current_training_data, lengths=lengths)
    
    # Save the trained model for the current word
    trained_model[word] = model


In [ ]:
# Extract features and labels for test clean data
x_test_clean, y_test_clean = zip(*[(feature, word) for word, data in test_clean_data.items() for feature in data]) 

# Extract features and labels for test noisy data
x_test_noisy, y_test_noisy = zip(*[(feature, word) for word, data in test_noisy_data.items() for feature in data]) 


In [ ]:
print(len(test_clean_data))
print(len(test_noisy_data))


In [ ]:
# Predictions for clean testing dataset
predictions_clean = [prediction(data, trained_model) for data in x_test_clean]

# Predictions for noisy testing dataset
predictions_noisy = [prediction(data, trained_model) for data in x_test_noisy]

# Obtaining the confusion matrix for the clean testing dataset
cm_clean = confusion_matrix(y_test_clean, predictions_clean, normalize="true")

# Obtaining the confusion matrix for the noisy testing dataset
cm_noisy = confusion_matrix(y_test_noisy, predictions_noisy, normalize="true")

classes = ['down', 'go', 'left', 'no', 'off', 'on', 'right', 'stop', 'up', 'yes']

In [ ]:
# Plotting confusion matrix for clean testing dataset and getting accuracy.
title_clean = 'Confusion Matrix for clean test and training data'
plot_confusion_matrix(cm_clean, classes, title_clean, "Clean_test_data.png")

accuracy_clean = np.diag(cm_clean).sum() / cm_clean.sum()
print(f'Percentage Accuracy for the trained model tested on the clean test dataset = {accuracy_clean * 100:.2f}%')

# Plotting confusion matrix for noisy testing dataset and getting accuracy.
title_noisy = 'Confusion Matrix for noisy test and training data'
plot_confusion_matrix(cm_noisy, classes, title_noisy, "Noisy_test_data.png")

accuracy_noisy = np.diag(cm_noisy).sum() / cm_noisy.sum()
print(f'Percentage Accuracy for the trained model tested on the noisy test dataset = {accuracy_noisy * 100:.2f}%')